# Partie une

## Praitraitement du texte

In [6]:
import re
from bs4 import BeautifulSoup
from urllib.request import urlopen

# Charger la page Wikipédia
url = "https://en.wikipedia.org/wiki/Rabat"
html = urlopen(url).read()

# Parser le HTML avec BeautifulSoup
soup = BeautifulSoup(html, "html.parser")

# Récupérer le contenu principal
content_div = soup.find("div", {"id": "bodyContent"})

# Extraire uniquement les paragraphes de texte
raw_text = " ".join(p.get_text() for p in content_div.find_all("p"))

# Nettoyage du texte :
# 1. Mettre en minuscules
# 2. Supprimer tout sauf les lettres latines et les espaces
clean_text = re.sub(r"[^a-z\s]", "", raw_text.lower())



# Afficher le texte nettoyé
print(clean_text)


rabat rbt also uk rbt us rbt arabic  romanized arrib is the capital city of morocco and the countrys seventhlargest city with an urban population of approximately   and a metropolitan population of over  million it is also the capital city of the rabatsalknitra administrative region rabat is located on the atlantic ocean at the mouth of the river bou regreg opposite sal the citys main commuter town
 rabat was founded in the th century by the almohads after a period of growth the city fell  into a long period of decline in the th century rabat became a haven for barbary pirates when the french established a protectorate over morocco in   rabat became its administrative center when morocco achieved independence in  rabat became its capital
 rabat temara and sal form a conurbation of over  million people rabat is one of four imperial cities of morocco and its medina is listed as a world heritage site it is accessible by train through the oncf system and by plane through the nearby rabatsa

## Construction de la matrice Stochastique

In [7]:
import numpy as np
import re
from collections import defaultdict

texte=clean_text

# Définir l'alphabet + espace (27 caractères)
alphabet = "abcdefghijklmnopqrstuvwxyz "
char_to_idx = {char: i for i, char in enumerate(alphabet)}

# Initialiser la matrice de transition (27x27)
M = np.zeros((27, 27))

# Construire la matrice des fréquences de transition
for i in range(len(texte) - 1):
    char1, char2 = texte[i], texte[i + 1]
    if char1 in char_to_idx and char2 in char_to_idx:
        M[char_to_idx[char1], char_to_idx[char2]] += 1

# Normalisation : Transformer en matrice stochastique
row_sums = M.sum(axis=1, keepdims=True)
P = np.divide(M, row_sums, where=row_sums != 0)  # Éviter la division par zéro

# Afficher la matrice de transition
print("Matrice de transition stochastique :\n", P)


Matrice de transition stochastique :
 [[9.43841435e-04 6.22935347e-02 2.50117980e-02 3.16186881e-02
  1.88768287e-03 1.51014630e-02 1.22699387e-02 1.88768287e-02
  3.16186881e-02 3.77536574e-03 3.30344502e-03 1.35441246e-01
  2.68994809e-02 1.66588013e-01 4.71920717e-04 1.03822558e-02
  2.35960359e-03 9.24964606e-02 9.62718263e-02 1.34025484e-01
  1.13260972e-02 1.03822558e-02 4.71920717e-03 9.43841435e-04
  1.98206701e-02 2.35960359e-03 7.88107598e-02]
 [3.44594595e-01 2.25225225e-03 6.75675676e-03 2.02702703e-02
  1.48648649e-01 0.00000000e+00 0.00000000e+00 2.25225225e-03
  5.40540541e-02 4.50450450e-03 0.00000000e+00 6.30630631e-02
  0.00000000e+00 6.75675676e-03 5.40540541e-02 0.00000000e+00
  0.00000000e+00 1.12612613e-02 1.12612613e-02 9.00900901e-03
  9.45945946e-02 0.00000000e+00 0.00000000e+00 0.00000000e+00
  9.23423423e-02 0.00000000e+00 7.43243243e-02]
 [1.55124654e-01 0.00000000e+00 6.23268698e-02 0.00000000e+00
  1.53739612e-01 4.15512465e-03 0.00000000e+00 1.01108033e-0

## Generer du texte

In [8]:
import numpy as np

def generate_text(P, alphabet, length=100, start_letter=None):
    """
    Génère un texte à partir d'une matrice de transition stochastique en sélectionnant toujours
    la lettre correspondant à la probabilité maximale.

    :param P: Matrice de transition stochastique (27x27)
    :param alphabet: Liste des caractères utilisés (lettres + espace)
    :param length: Nombre de caractères à générer
    :param start_letter: Lettre de départ (None pour choisir aléatoirement)
    :return: Texte généré
    """
    # Dictionnaire pour associer chaque lettre à son indice
    char_to_idx = {char: i for i, char in enumerate(alphabet)}
    idx_to_char = {i: char for i, char in enumerate(alphabet)}

    # Choisir une lettre de départ
    if start_letter is None or start_letter not in char_to_idx:
        current_idx = np.random.choice(len(alphabet))  # Aléatoire
    else:
        current_idx = char_to_idx[start_letter]  # Fixé

    generated_text = idx_to_char[current_idx]

    # Génération caractère par caractère
    for _ in range(length - 1):
        next_idx = np.argmax(P[current_idx])  # Trouver l'indice de la colonne avec la probabilité maximale
        generated_text += idx_to_char[next_idx]
        current_idx = next_idx  # Mise à jour de current_idx

    return generated_text


# Génération d'un texte de 200 caractères à partir de 'r'
generated_text = generate_text(P, alphabet, length=200, start_letter="f")
print("Texte généré :\n", generated_text)


Texte généré :
 f the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the th


## Evaluation du modele

In [10]:
import numpy as np

def evaluate_markov_model(P, alphabet, text):
    """
    Évalue la performance du modèle de Markov en comparant les probabilités de transition
    d'un premier texte avec celles du second texte.

    :param P: Matrice de transition stochastique (27x27)
    :param alphabet: Liste des caractères utilisés (lettres + espace)
    :param text: Texte de test pour l'évaluation
    :return: Score final et précision globale
    """
    # Dictionnaire pour convertir les caractères en indices
    char_to_idx = {char: i for i, char in enumerate(alphabet)}

    # Initialisation des scores et tentatives
    score = 0
    tentatives = 0

    # Parcours du texte (on s'arrête à l'avant-dernière lettre)
    for i in range(len(text) - 1):
        x = text[i]   # Lettre actuelle
        y = text[i+1] # Lettre suivante

        # Vérifier si x et y sont dans l'alphabet (sinon, ignorer)
        if x in char_to_idx and y in char_to_idx:
            x_idx = char_to_idx[x]
            y_idx = char_to_idx[y]

            # Ajouter la probabilité de transition au score
            score += P[x_idx, y_idx]
            tentatives += 1

    # Calcul de la précision globale
    precision = score / tentatives if tentatives > 0 else 0
    return score, precision



text = generated_text

# Évaluation du modèle
score, precision = evaluate_markov_model(P, alphabet, text)
print(f"Score total: {score:.4f}")
print(f"Précision globale: {precision:.4%}")


Score total: 68.3811
Précision globale: 34.3624%


# Etape 3

### Modele de Markov a la base des tripplets

##### Construction de la matrice de transition

In [11]:
import numpy as np
from collections import defaultdict

def build_transition_matrix(text, alphabet):
    """
    Construit une matrice de transition basée sur les triplets de lettres.

    :param text: Texte source pour construire la matrice
    :param alphabet: Alphabet utilisé (26 lettres + espace)
    :return: Matrice de transition normalisée (P) et dictionnaires d'index
    """
    n = len(alphabet)
    triplet_to_idx = {triplet: i for i, triplet in enumerate(
        [a+b+c for a in alphabet for b in alphabet for c in alphabet])}
    char_to_idx = {char: i for i, char in enumerate(alphabet)}

    # Initialisation de la matrice de transition
    P = np.zeros((n**3, n))

    # Comptage des occurrences des triplets suivis d'une lettre
    for i in range(len(text) - 3):
        triplet = text[i:i+3]
        next_char = text[i+3]
        if triplet in triplet_to_idx and next_char in char_to_idx:
            P[triplet_to_idx[triplet], char_to_idx[next_char]] += 1

    # Normalisation (transformer en probabilités)
    P = P / P.sum(axis=1, keepdims=True)
    P = np.nan_to_num(P)  # Remplace les NaN par 0 (cas où aucune transition observée)

    return P, triplet_to_idx, char_to_idx



###### Generation du texte et Evaluation du modele

Fonction generant du texte a partir du modele de markov

In [12]:
def generate_text(P, triplet_to_idx, char_to_idx, alphabet, length=100, start_triplet=None):
    """
    Génère un texte en suivant la matrice de transition des triplets.

    :param P: Matrice de transition
    :param triplet_to_idx: Dictionnaire mappant les triplets à des indices
    :param char_to_idx: Dictionnaire mappant les lettres à des indices
    :param alphabet: Alphabet utilisé
    :param length: Longueur du texte à générer
    :param start_triplet: Triplet de départ (None pour un choix aléatoire)
    :return: Texte généré
    """
    idx_to_char = {i: char for char, i in char_to_idx.items()}
    idx_to_triplet = {i: triplet for triplet, i in triplet_to_idx.items()}

    # Choisir un triplet de départ
    if start_triplet is None or start_triplet not in triplet_to_idx:
        start_idx = np.random.choice(len(triplet_to_idx))  # Aléatoire
        current_triplet = idx_to_triplet[start_idx]
    else:
        current_triplet = start_triplet

    generated_text = current_triplet

    # Génération de caractères
    for _ in range(length - 3):
        triplet_idx = triplet_to_idx[current_triplet]
        next_idx = np.argmax(P[triplet_idx])  # Choix de la lettre avec la plus haute proba
        next_char = idx_to_char[next_idx]
        generated_text += next_char
        current_triplet = current_triplet[1:] + next_char  # Met à jour le triplet

    return generated_text



Methode d evaluation du modele

In [15]:
import numpy as np

def evaluate_markov_model_triplets(P, text):
    """
    Évalue la performance du modèle de Markov basé sur les triplets de lettres.

    :param P: Matrice de transition stochastique (27³ x 27³)
    :param triplets: Liste des triplets possibles
    :param text: Texte de test pour l'évaluation
    :return: Score final et précision globale
    """

    score = 0
    tentatives = 0

    for i in range(len(text) - 3):
        triplet_x = text[i:i+3]  # Triplet actuel
        next_char = text[i+3]


        if triplet_x in triplet_to_idx :
            x_idx = triplet_to_idx[triplet_x]
            y_idx = char_to_idx[next_char]

            score += P[x_idx, y_idx]
            tentatives += 1

    precision = score / tentatives if tentatives > 0 else 0
    return score, precision




Test du modele

In [19]:

# Construction de la matrice de transition
P, triplet_to_idx, char_to_idx = build_transition_matrix(texte, alphabet)

epsilon = 1e-8  # Petit terme d'évitement du zéro
P_epsilon = (P + epsilon) / (P.sum(axis=1, keepdims=True) + epsilon * P.shape[1])


# Génération d'un texte de 200 caractères
generated_text_2 = generate_text(P_epsilon, triplet_to_idx, char_to_idx, alphabet, length=200, start_triplet="rab")

## Exemple d'évaluation
score, precision = evaluate_markov_model_triplets(P_epsilon, generated_text_2)


print("Texte généré :\n", generated_text_2)
print(f"Score total (triplets) : {score:.4f}")
print(f"Précision globale (triplets) : {precision:.4%}")

Texte généré :
 rabat the almohad and the almohad and the almohad and the almohad and the almohad and the almohad and the almohad and the almohad and the almohad and the almohad and the almohad and the almohad and th
Score total (triplets) : 115.9826
Précision globale (triplets) : 58.8744%


<ipython-input-11-4a6cfcfa839a>:28: RuntimeWarning: invalid value encountered in divide
  P = P / P.sum(axis=1, keepdims=True)


# Partie3

## Modele de Markov a la base de transition stochastique entre mots

In [27]:
import numpy as np
import re
from collections import defaultdict

def preprocess_text(text):
    """ Nettoie le texte et extrait les mots en minuscules. """
    words = re.findall(r'\b\w+\b', text.lower())  # Extraction des mots
    return words

def build_transition_matrix(text):
    """ Construit la matrice de transition entre les mots du texte. """
    words = preprocess_text(text)
    unique_words = list(set(words))  # Liste des mots uniques
    word_to_idx = {word: i for i, word in enumerate(unique_words)}
    idx_to_word = {i: word for word, i in word_to_idx.items()}

    N = len(unique_words)
    P = np.zeros((N, N))  # Matrice de transition

    # Remplissage de la matrice avec les fréquences de transition
    for i in range(len(words) - 1):
        current_idx = word_to_idx[words[i]]
        next_idx = word_to_idx[words[i + 1]]
        P[current_idx, next_idx] += 1

    # Normalisation pour obtenir des probabilités
    row_sums = P.sum(axis=1, keepdims=True)
    P = np.divide(P, row_sums, where=row_sums != 0)  # Évite la division par zéro

    return P, word_to_idx, idx_to_word

def generate_text(P, word_to_idx, idx_to_word, length=50, start_word=None):
    """ Génère un texte en utilisant la matrice de transition. """
    if start_word is None or start_word not in word_to_idx:
        current_idx = np.random.choice(len(word_to_idx))  # Choix aléatoire
    else:
        current_idx = word_to_idx[start_word]

    generated_text = [idx_to_word[current_idx]]

    for _ in range(length - 1):
        next_idx = np.random.choice(len(word_to_idx), p=P[current_idx])
        generated_text.append(idx_to_word[next_idx])
        current_idx = next_idx  # Mise à jour du mot courant

    return ' '.join(generated_text)





#### Generation du texte

In [28]:
P, word_to_idx, idx_to_word = build_transition_matrix(texte)
generated_text = generate_text(P, word_to_idx, idx_to_word, length=200, start_word="the")

print("Texte généré :\n", generated_text)

Texte généré :
 the population of the late th centuries the new fortified imperial residence built a military tribe military staging ground of which hosts the romans on the kasbah as landmarks from cool ranging between and bab chellah a pirate attackcitation needed during the medina located below the completion of sala or oudayas museum of old sal whereas the medina also responsible for almohad caliph abu alhasan as of french government of the united states had established in the consuls became an average annual precipitation of the region under the kasbah to design the youssoufia region in with exhibits focusing on other independent art deco and bab udaya or political contacts between sala or the site of what is a controversy that only were relatively small census figures are the association football matches and souissi are not been fez hr on such as the mosques and the zoo has a summit recognized the shores of this almohad walls and with the th century the new sal can also uk rbt ara

###### Methode  d **Evaluation**

In [29]:
import numpy as np

def evaluate_markov_model_words(P, word_to_idx, text):
    """
    Évalue la performance du modèle de Markov basé sur les transitions entre mots.

    :param P: Matrice de transition stochastique (N x N) où N est la taille du vocabulaire.
    :param word_to_idx: Dictionnaire associant chaque mot à un indice unique.
    :param text: Texte de test pour l'évaluation.
    :return: Score total et précision globale.
    """
    words = text.split()  # Tokenisation en mots

    score = 0
    tentatives = 0
    vocab_size = len(word_to_idx)

    for i in range(len(words) - 1):
        word_x = words[i]   # Mot actuel
        word_y = words[i+1] # Mot suivant

        if word_x in word_to_idx and word_y in word_to_idx:
            x_idx = word_to_idx[word_x]
            y_idx = word_to_idx[word_y]

            if x_idx < vocab_size and y_idx < vocab_size:  # Vérification des indices
                score += P[x_idx, y_idx]
                tentatives += 1

    precision = score / tentatives if tentatives > 0 else 0
    return score, precision


# Évaluation du modèle avec le texte généré
score, precision = evaluate_markov_model_words(P, word_to_idx, generated_text)
print(f"Score total : {score:.4f}")
print(f"Précision globale : {precision:.4%}")


Score total : 66.9682
Précision globale : 33.6524%


In [30]:
P, word_to_idx, idx_to_word = build_transition_matrix(texte)
generated_text = generate_text(P, word_to_idx, idx_to_word, length=20, start_word="the")

print("Texte généré :\n", generated_text)

Texte généré :
 the former almohad kasbah attracting between and marrakesh while were hosted by sultan moulay abdellah stadium in making it was


In [31]:
score, precision = evaluate_markov_model_words(P, word_to_idx, generated_text)
print(f"Score total : {score:.4f}")
print(f"Précision globale : {precision:.4%}")

Score total : 6.0420
Précision globale : 31.7998%
